# Los tipos de datos faltantes en Series de Tiempo

Las técnicas de manejo de datos faltantes dependerán del tipo de datos faltantes que tengamos en nuestra Serie de Tiempo.

Y en esencia podremos tener datos faltantes en las observaciones, en la variable temporal (es decir en las marcas de tiempo) o una combinación de las dos anteriores.

Así que en esta lección veremos los cuatro tipos de datos faltantes que podemos tener en una Serie de Tiempo así como las técnicas que podemos usar para detectar estos datos faltantes.

Entender y saber detectar estos tipos de datos faltantes nos permitirá más adelante decidir cuál técnica usar para realizar el manejo correspondiente.

## 1. Los sets de datos a usar

Para entender los diferentes tipos de datos faltantes que podemos tener en una Serie de Tiempo haremos uso de estos sets de datos:

- *co2_faltantes.csv*: emisiones anuales (1750-2020) de dióxido de carbono  a la atmósfera provenientes de la quema de combustibles fósiles
- *clicks-faltantes-marcas.csv*, *clicks-faltantes-marcas-ocultas.csv* y *clicks-faltantes-multiples.csv*: información de interacción de usuarios (número de clicks) de un sitio web de artículos de vestuario.

Leamos entonces estos sets de datos:

In [1]:
import pandas as pd

# Leer datasets
RUTA = '/Users/miguel/Library/CloudStorage/GoogleDrive-miguel@codificandobits.com/My Drive/02-CODIFICANDOBITS.COM/04-Academia/01-Cursos/23-2024-05-SeriesDeTiempo-Preprocesamiento/data/'

co2_df = pd.read_csv(RUTA + 'co2_faltantes.csv',
                    parse_dates = ['año'],
                    index_col = ['año'])

clicks_df = pd.read_csv(RUTA + 'clicks_faltantes_marcas.csv',
                    parse_dates = ['fecha'],
                    index_col = ['fecha'])

clicks_oc_df = pd.read_csv(RUTA + 'clicks_faltantes_marcas_ocultas.csv',
                    parse_dates = ['fecha'],
                    index_col = ['fecha'])

clicks_mult_df = pd.read_csv(RUTA + 'clicks_faltantes_multiples.csv',
                             parse_dates = ['fecha'],
                             index_col = ['fecha'])

Muy bien, ahora sí analicemos en detalle los diferentes tipos de datos faltantes que podremos encontrar en una Serie de Tiempo.

## Situación 1: observaciones faltantes

En este caso nuestra Serie de Tiempo tendrá las marcas de tiempo (el índice del *DataFrame*) completas pero la(s) columna(s) (es decir las observaciones) contendrán datos faltantes.

Cuando la columna contiene valores numéricos, el valor asignado a la(s) celda(s) faltante(s) será *NaN* (*Not A Number*).

Veamos por ejemplo la serie "co2_df":

In [2]:
co2_df

,co2
año,
1750-01-01,0.0125
1760-01-01,0.0128
1770-01-01,NaN
1780-01-01,0.0169
1790-01-01,0.0206
...,...
2016-01-01,4.7496
2017-01-01,4.7595
2018-01-01,4.8022


Vemos que el tercer elemento de la columna "co2" contiene un dato faltante, marcado como NaN. Esto es un ejemplo de una observación faltante.

### Detección de observaciones faltantes

La forma más sencilla de detectar las observaciones faltantes es usando el método "isna()". Por ejemplo, si encadenamos este método al método "sum()" podremos realizar un conteo de datos faltantes en la(s) columna(s) de interés:

In [3]:
co2_df.isna().sum()

co2    26
dtype: int64

Vemos que en total la columna "co2" contiene 26 datos faltantes.

Si nos interesa podemos incluso determinar cuáles son los índices con observaciones faltantes:

In [4]:
ind_falt_co2 = co2_df[co2_df['co2'].isna()].index
ind_falt_co2

DatetimeIndex(['1770-01-01', '1927-01-01', '1928-01-01', '1929-01-01',
               '1930-01-01', '1931-01-01', '1932-01-01', '1933-01-01',
               '1934-01-01', '1935-01-01', '1936-01-01', '1937-01-01',
               '1974-01-01', '1975-01-01', '1976-01-01', '1977-01-01',
               '1978-01-01', '1979-01-01', '1980-01-01', '1981-01-01',
               '1982-01-01', '1983-01-01', '1984-01-01', '1985-01-01',
               '1986-01-01', '1987-01-01'],
              dtype='datetime64[ns]', name='año', freq=None)

Más adelante, en las próximas lecciones, veremos cómo realizar el manejo de estos datos faltantes.

## Situación 2: marcas de tiempo faltantes

El segundo tipo de dato faltante que podemos tener en una Serie de Tiempo es cuando las observaciones están completas pero las marcas de tiempo (el índice del *DataFrame*) están incompletas.

Por ejemplo, veamos el *DataFrame* *clicks_df*:

In [5]:
clicks_df

,precio,ubicación,clicks
fecha,,,
2008-04-01,43.155647,2,18784.0
2008-04-02,43.079056,1,24738.0
NaT,43.842609,2,15209.0
2008-04-04,43.382794,2,14320.0
2008-04-05,43.941176,1,11974.0
...,...,...,...
2008-08-09,44.182033,1,6716.0
2008-08-10,43.608260,1,9523.0
2008-08-11,43.553363,1,8881.0


En este caso, si usamos el método anterior (combinación de los métodos "isna()" y "sum()") encontraremos que las columnas "precio" y "ubicación" están completas y que hay 16 datos incompletos en "clicks":

In [6]:
clicks_df.isna().sum()

precio        0
ubicación     0
clicks       16
dtype: int64

Sin embargo, **nada nos garantiza que no tengamos datos faltantes en el índice de la serie**.

De hecho, podemos ver que después de la segunda fila (fecha: 2008-04-02) tenemos un "salto" hasta el 2008-04-04. Es decir que al menos hay una marca de tiempo faltante: 2008-04-03.

Esta marca de tiempo faltante es etiquetada con un valor *NaT* (*Not a Time*)por Pandas.

### Detección de marcas de tiempo faltantes

Podemos usar una variantes del método anterior ("isna()" combinado con "sum()") para detectar marcas de tiempo (tipo *NaT*) faltantes. Simplemente extraemos primero el índice del *DataFrame* y luego si usamos los dos métodos anteriores:

In [7]:
clicks_df.index.isna().sum()

2

Lo anterior quiere decir que el índice de la serie contiene 2 valores faltantes.

## Situación 3: marcas de tiempo faltantes "ocultas"

Esta situación es similar a la anterior. Sin embargo, en principio ninguna marca de tiempo aparece etiquetada como *NaT*.

Por ejemplo veamos lo que ocurre con el *DataFrame* *clicks_oc_df*:

In [8]:
clicks_oc_df

,precio,ubicación,clicks
fecha,,,
2008-04-01,43.155647,2,18784.0
2008-04-02,43.079056,1,24738.0
2008-04-03,43.842609,2,15209.0
2008-04-05,43.941176,1,11974.0
2008-04-06,44.403936,1,11007.0
...,...,...,...
2008-08-09,44.182033,1,6716.0
2008-08-10,43.608260,1,9523.0
2008-08-11,43.553363,1,8881.0


Detectemos observaciones faltantes:

In [9]:
clicks_oc_df.isna().sum()

precio        0
ubicación     0
clicks       16
dtype: int64

Y ahora detectemos marcas de tiempo faltantes:

In [10]:
clicks_oc_df.index.isna().sum()

0

Al parecer no hay marcas de tiempo faltantes. Sin embargo, veamos nuevamente el índice de este *DataFrame*:

In [11]:
clicks_oc_df

,precio,ubicación,clicks
fecha,,,
2008-04-01,43.155647,2,18784.0
2008-04-02,43.079056,1,24738.0
2008-04-03,43.842609,2,15209.0
2008-04-05,43.941176,1,11974.0
2008-04-06,44.403936,1,11007.0
...,...,...,...
2008-08-09,44.182033,1,6716.0
2008-08-10,43.608260,1,9523.0
2008-08-11,43.553363,1,8881.0


Vemos que en principio hay un "salto" entre 2008-04-03 y 2008-04-05. Es decir que la Serie de Tiempo tiene una frecuencia diaria pero esta frecuencia no se mantiene para 2008-04-04.

En este caso Pandas lee la Serie de Tiempo de manera correcta y **no aparecerán marcas faltantes etiquetadas con *NaT***.

### Detección usando el método "difference"

En este caso la idea es:

1. Crear un rango ideal de fechas
2. Usar "difference" para encontrar las diferencias entre este rango y el índice original del DataFrame

Veamos esta implementación:

In [12]:
# 1. Crear un rango completo de fechas
rango_fechas = pd.date_range(start=clicks_oc_df.index.min(), end=clicks_oc_df.index.max(), freq='D')
rango_fechas

DatetimeIndex(['2008-04-01', '2008-04-02', '2008-04-03', '2008-04-04',
               '2008-04-05', '2008-04-06', '2008-04-07', '2008-04-08',
               '2008-04-09', '2008-04-10',
               ...
               '2008-08-04', '2008-08-05', '2008-08-06', '2008-08-07',
               '2008-08-08', '2008-08-09', '2008-08-10', '2008-08-11',
               '2008-08-12', '2008-08-13'],
              dtype='datetime64[ns]', length=135, freq='D')

In [13]:
# 2. Usar "difference" para encontrar las diferencias entre este rango y el índice original del *DataFrame*
rango_fechas.difference(clicks_oc_df.index)

DatetimeIndex(['2008-04-04', '2008-04-07', '2008-04-09'], dtype='datetime64[ns]', freq=None)

## Situación 4: observaciones y marcas de tiempo faltantes

En este caso tendremos tanto observaciones faltantes (marcadas como *NaN*) como marcas de tiempo faltantes (marcadas o no como *NaT*).

Por ejemplo veamos el *DataFrame* *clicks_mult_df*:

In [14]:
clicks_mult_df

,precio,ubicación,clicks
fecha,,,
2008-04-01,43.155647,2.0,18784
2008-04-02,43.079056,1.0,24738
NaT,43.842609,NaN,15209
NaT,NaN,1.0,14018
NaT,43.941176,1.0,11974
...,...,...,...
2008-08-09,44.182033,1.0,6716
2008-08-10,43.608260,1.0,9523
2008-08-11,43.553363,1.0,8881


Y en este caso podemos simplemente combinar los métodos anteriores para detectar las observaciones y las marcas de tiempo faltantes.

Para el caso de las observaciones:

In [15]:
clicks_mult_df.isna().sum()

precio        1
ubicación     1
clicks       14
dtype: int64

Y para el caso de las marcas de tiempo "NaT":

In [16]:
clicks_mult_df.index.isna().sum()

4

En este caso es importante tener en cuenta que además de las marcas de tiempo faltantes marcadas como *NaT* podríamos tener marcas de tiempo ocultas. Pero en este caso deberíamos primero manejar las marcas *NaT* y luego determinar si existen o no marcas ocultas (de esto hablaremos en detalle en la próxima lección).